In [ ]:
import anndata
from scdesigner.minimal import NegBinCopula

example_sce = anndata.read_h5ad("data/example_sce.h5ad")
sim = NegBinCopula(mean_formula="~ bs(pseudotime, df=3, degree=1)", dispersion_formula="~ 1", copula_formula="~ -1 + cell_type")
sim.fit(example_sce, max_epochs=50, lr=.01, batch_size=3000)

In [ ]:
print(sim.parameters)
print(sim.sample())
print(sim.predict())

In [ ]:
import altair as alt
import numpy as np
import pandas as pd

plot_df = pd.DataFrame({
    "time": np.concatenate([example_sce.obs['pseudotime'], example_sce.obs['pseudotime']]),
    "value": np.concatenate([np.array(example_sce.X.todense())[:,50], sim.sample().X[:, 50]]),
    "type": ["real"] * len(example_sce) + ["sim"] * len(example_sce)
})

alt.Chart(plot_df)\
    .mark_circle()\
    .encode(x="time", y="value", color="type")

In [ ]:
from scdesigner.minimal import ZeroInflatedNegBinCopula

sim = ZeroInflatedNegBinCopula("~ bs(pseudotime, df=5)", "~ 1", "~ pseudotime", "~ -1 + cell_type")
sim.fit(example_sce, max_epochs=1, lr=0.01)

In [ ]:
print(sim.parameters)
print(sim.sample())
print(sim.predict())

In [ ]:
from scdesigner.minimal import BernoulliCopula

sim = BernoulliCopula("~ bs(pseudotime, df=5)", "~ -1 + cell_type")
sim.fit(example_sce, max_epochs=1, lr=0.01)

print(sim.parameters)
print(sim.sample())
print(sim.predict())

In [ ]:
from scdesigner.minimal import GaussianCopula

sim = GaussianCopula("~ bs(pseudotime, df=5)", "~ pseudotime", "~ -1 + cell_type")
sim.fit(example_sce, max_epochs=1, lr=0.01)

print(sim.parameters)
print(sim.sample())
print(sim.predict())

In [ ]:
from scdesigner.minimal.negbin import NegBin
from scdesigner.minimal import CompositeCopula

genes = example_sce.var_names
specification = [(genes[:10], NegBin("~ bs(pseudotime, df=5)")), (genes[10:20], NegBin("~ 1"))]
# sim = CompositeCopula(specification, "~ 1")
# sim.fit(example_sce, max_epochs=1)
# sim.predict(example_sce.obs[:10])
# sim.sample(example_sce.obs[:10])
# sim.complexity()

In [ ]:
preds = sim.predict(example_sce.obs[:10])

{k: v[:4, :4] for k, v in preds.items()}

In [ ]:
sim.sample(example_sce.obs[:10])

In [ ]:
sim.complexity()

In [ ]:
sim.parameters

In [ ]:
import scdesigner.minimal.transform as tr

null_sim = tr.nullify(sim, "pseudotime", "Pyy|Iapp", "mean")
null_sim.marginal.predict.coefs["mean"][:3, :4]

In [ ]:
sim.parameters["copula"]['cell_type[Ngn3 low EP]'].cov

In [ ]:
import copy

amplified_sim = tr.amplify(sim, 2, "pseudotime", "Pyy|Iapp", "mean")
decorr_sim = tr.decorrelate(sim, "Pyy|Iapp", "Pyy|Iapp", "cell_type[Ngn3 low EP]")
corr_sim = tr.correlate(sim, 2, "Pyy|Iapp", "Pyy|Iapp", "cell_type[Ngn3 low EP]")
decorr_sim.parameters["copula"]["cell_type[Ngn3 low EP]"].cov.iloc[:4, :4]

In [ ]:
sim.parameters["copula"]["cell_type[Ngn3 low EP]"].cov.iloc[:4, :4]

In [ ]:
# Also can be done by calling the method directly on the copula object
decorr_sim = copy.deepcopy(sim)
decorr_sim.copula.decorrelate("Pyy|Iapp", "Pyy|Iapp", "cell_type[Ngn3 low EP]")
corr_sim = copy.deepcopy(sim)
corr_sim.copula.correlate(2, "Pyy|Iapp", "Pyy|Iapp", "cell_type[Ngn3 low EP]")
decorr_sim.parameters["copula"]["cell_type[Ngn3 low EP]"].cov.iloc[:4, :4]

Here is a more manual but very general way to substitute parameters.

In [ ]:
import pandas as pd
import numpy as np

new_param = np.random.normal(size=(6, 100))
sub_sim_mean = tr.replace_param(sim, ["marginal", "mean"], new_param)

new_covariance = 0.9 * np.eye(100) + 0.1 * np.ones((100, 100))
new_covariance = pd.DataFrame(new_covariance, columns=example_sce.var_names, index=example_sce.var_names)
sub_sim_cov = tr.replace_param(sim, ["copula", "cell_type[Ngn3 low EP]"], new_covariance)

In [ ]:
sim.parameters["copula"]["cell_type[Ngn3 low EP]"].cov.iloc[:4, :4]

In [ ]:
sub_sim_cov.parameters["copula"]["cell_type[Ngn3 low EP]"].cov.iloc[:4, :4]

Here's an example showing how dispersion can be modeled.

In [ ]:
sim = NegBinCopula("~ bs(pseudotime,df=5)", "~ pseudotime", "~ -1 + cell_type")
sim.fit(example_sce, batch_size=int(256), max_epochs=1)

In [ ]:
print(sim.parameters)
print(sim.sample())
print(sim.predict())

In [ ]:
# example_sce = anndata.read_h5ad("data/million_cells.h5ad", backed=True, chunk_size=int(2e4))

# sim = NegBinCopula("~ celltype", "~ 1", "~ -1 + celltype")
#sim.fit(example_sce, batch_size=int(1000), lr=0.01)